In [27]:
from pyecharts.globals import CurrentConfig, NotebookType
# CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

In [3]:
import pandas as pd
from pyecharts.charts import Line
from pyecharts import options as opts
from IPython.display import display, HTML
import ipywidgets as widgets
import io

# 全局变量
data_raw = pd.DataFrame()
data_df = pd.DataFrame()
header_row_index = 0
data_end_row_index = 0

# 上传按钮
upload_button = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=False,
    description='上传表格',
    button_style='info'
)

# 标题行 / 结束行选择
start_row_dropdown = widgets.Dropdown(description="标题行：")
end_row_dropdown = widgets.Dropdown(description="结束行：")

# x轴 和 y轴选择框
x_dropdown = widgets.Dropdown(description="X 轴：")
y_dropdowns = [widgets.Dropdown(description=f"Y 轴 {i+1}：") for i in range(5)]

# 输出区域
output_chart = widgets.Output()

# 处理上传文件
def on_upload_change(change):
    global data_raw
    output_chart.clear_output()

    if upload_button.value:
        # uploaded_file = list(upload_button.value.values())[0]
        # content = uploaded_file['content']
        # filename = uploaded_file['metadata']['name']

        # 获取上传文件内容（兼容 ipywidgets 7.x 和 8.x）
        if isinstance(upload_button.value, dict):  # ipywidgets 7.x
            uploaded_file = next(iter(upload_button.value.values()))
            filename = list(upload_button.value.keys())[0]
            content = uploaded_file['content']
        elif isinstance(upload_button.value, tuple):  # ipywidgets 8.x
            uploaded_file = upload_button.value[0]
            filename = uploaded_file['name']
            content = uploaded_file['content']
        else:
            with output_chart:
                print("无法识别上传文件格式")
                return

        try:
            if filename.endswith('.csv'):
                data_raw = pd.read_csv(io.BytesIO(content), header=None)
            elif filename.endswith('.xlsx'):
                data_raw = pd.read_excel(io.BytesIO(content), header=None, engine='openpyxl')
        except Exception as e:
            with output_chart:
                print("读取失败:", e)
            return

        row_count = len(data_raw)
        options = [(f"第 {i+1} 行", i) for i in range(row_count)]
        start_row_dropdown.options = options
        start_row_dropdown.value = 0  # 默认选择第一行作为标题行
        end_row_dropdown.options = options
        end_row_dropdown.value = row_count - 1  # 下标从 0 开始，所以 -1 才是最后一行

        with output_chart:
            print("文件读取成功，共", row_count, "行")

upload_button.observe(on_upload_change, names='value')

# 更新列名下拉选框
def update_column_selectors():
    global data_df

    try:
        header_index = start_row_dropdown.value
        end_index = end_row_dropdown.value

        if header_index is None or end_index is None:
            return

        if header_index >= end_index:
            with output_chart:
                print("标题行必须在结束行之前")
            return

        # 提取标题和数据
        headers = data_raw.iloc[header_index].tolist()
        data_df = data_raw.iloc[header_index + 1: end_index + 1].copy()
        data_df.columns = headers
        data_df.reset_index(drop=True, inplace=True)

        # 更新下拉框选项
        x_dropdown.options = headers
        for yd in y_dropdowns:
            yd.options = [""] + headers

        update_chart()

    except Exception as e:
        with output_chart:
            print("更新失败:", e)

start_row_dropdown.observe(lambda x: update_column_selectors(), names='value')
end_row_dropdown.observe(lambda x: update_column_selectors(), names='value')

# 绘制图表
def update_chart(*args):
    output_chart.clear_output()
    if data_df.empty:
        return

    x_col = x_dropdown.value
    y_cols = [yd.value for yd in y_dropdowns if yd.value]

    if not x_col or not y_cols:
        return

    try:
        x_data = data_df[x_col].astype(str).tolist()
        line = Line(init_opts=opts.InitOpts(width="100%", height="600px")).add_xaxis(x_data)
        # line = Line(init_opts=opts.InitOpts(width="100%", height="100%")).add_xaxis(x_data)

        for y in y_cols:
            y_data = data_df[y].astype(float).tolist()
            line.add_yaxis(y, y_data)

        line.set_global_opts(
            title_opts=opts.TitleOpts(title="曲线图"),
            tooltip_opts=opts.TooltipOpts(trigger="axis"),
            toolbox_opts=opts.ToolboxOpts(),
            datazoom_opts=[
                opts.DataZoomOpts(type_="slider"),     # 底部滑块
                opts.DataZoomOpts(type_="inside")      # 鼠标滚轮缩放（内嵌）
            ],
            
        )



        # with output_chart:
        #     # display(HTML(line.render_embed()))
        #     line.render_notebook()
        with output_chart:
            display(line.render_notebook())

    except Exception as e:
        with output_chart:
            print("绘图失败：", e)

# 给下拉绑定图表刷新事件
x_dropdown.observe(update_chart, names='value')
for yd in y_dropdowns:
    yd.observe(update_chart, names='value')

# 显示控件
display(upload_button)
display(widgets.HBox([start_row_dropdown, end_row_dropdown]))
display(widgets.HBox([x_dropdown]))
display(widgets.HBox(y_dropdowns))
display(output_chart)


FileUpload(value=(), accept='.csv,.xlsx', button_style='info', description='上传表格')

Output()

In [6]:
import pandas as pd
import io
import csv

from pyecharts.charts import Line
from pyecharts import options as opts
from IPython.display import display, HTML
import ipywidgets as widgets

# 全局变量
data_raw = pd.DataFrame()
data_df = pd.DataFrame()

# 上传按钮
upload_button = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=False,
    description='上传表格',
    button_style='info'
)

# 标题行 / 结束行选择
start_row_dropdown = widgets.Dropdown(description="标题行：")
end_row_dropdown = widgets.Dropdown(description="结束行：")

# x轴 和 y轴选择器
x_dropdown = widgets.Dropdown(description="X 轴：")
y_select = widgets.SelectMultiple(description="Y 轴：", layout=widgets.Layout(height='150px'))

# 输出区域
output_chart = widgets.Output()

# 宽容 CSV 读取函数
def read_csv_loose(content_bytes):
    try:
        decoded_text = content_bytes.decode('utf-8')
    except UnicodeDecodeError:
        decoded_text = content_bytes.decode('gbk', errors='ignore')

    reader = csv.reader(io.StringIO(decoded_text))
    rows = list(reader)
    max_cols = max(len(r) for r in rows)
    normalized = [r + [''] * (max_cols - len(r)) for r in rows]
    return pd.DataFrame(normalized)

# 上传文件响应
def on_upload_change(change):
    global data_raw
    output_chart.clear_output()

    if upload_button.value:
        try:
            if isinstance(upload_button.value, dict):  # ipywidgets 7.x
                uploaded_file = next(iter(upload_button.value.values()))
                filename = list(upload_button.value.keys())[0]
                content = uploaded_file['content']
            else:  # ipywidgets 8.x
                uploaded_file = upload_button.value[0]
                filename = uploaded_file['name']
                content = uploaded_file['content']
        except Exception as e:
            with output_chart:
                print("无法读取上传内容：", e)
            return

        try:
            if filename.endswith('.csv'):
                if isinstance(content, memoryview):
                    content = content.tobytes()
                data_raw = read_csv_loose(content)


            # if filename.endswith('.csv'):
                data_raw = read_csv_loose(content)
            elif filename.endswith('.xlsx'):
                data_raw = pd.read_excel(io.BytesIO(content), header=None, engine='openpyxl')
            else:
                with output_chart:
                    print("不支持的文件格式")
                return
        except Exception as e:
            with output_chart:
                print("读取失败:", e)
            return

        # 更新行选择器
        row_count = len(data_raw)
        options = [(f"第 {i+1} 行", i) for i in range(row_count)]
        start_row_dropdown.options = options
        start_row_dropdown.value = 0
        end_row_dropdown.options = options
        end_row_dropdown.value = row_count - 1

        with output_chart:
            print("文件读取成功，共", row_count, "行")

upload_button.observe(on_upload_change, names='value')

# 刷新列选择控件
def update_column_selectors(*args):
    global data_df

    output_chart.clear_output()
    try:
        header_index = start_row_dropdown.value
        end_index = end_row_dropdown.value

        if header_index is None or end_index is None or header_index >= end_index:
            with output_chart:
                print("标题行设置错误")
            return

        headers = data_raw.iloc[header_index].tolist()
        data_df = data_raw.iloc[header_index + 1: end_index + 1].copy()
        data_df.columns = headers
        data_df.reset_index(drop=True, inplace=True)

        x_dropdown.options = headers
        y_select.options = headers

        update_chart()

    except Exception as e:
        with output_chart:
            print("更新列选择器失败：", e)

start_row_dropdown.observe(update_column_selectors, names='value')
end_row_dropdown.observe(update_column_selectors, names='value')

# 绘制图表
def update_chart(*args):
    output_chart.clear_output()

    if data_df.empty:
        return

    x_col = x_dropdown.value
    y_cols = list(y_select.value)

    if not x_col or not y_cols:
        return

    try:
        x_data = data_df[x_col].astype(str).tolist()
        line = Line(init_opts=opts.InitOpts(width="100%", height="600px")).add_xaxis(x_data)

        for y in y_cols:
            y_data = pd.to_numeric(data_df[y], errors='coerce').fillna(0).tolist()
            line.add_yaxis(y, y_data)

        line.set_global_opts(
            title_opts=opts.TitleOpts(title="曲线图"),
            tooltip_opts=opts.TooltipOpts(trigger="axis"),
            toolbox_opts=opts.ToolboxOpts(),
            datazoom_opts=[
                opts.DataZoomOpts(type_="slider"),
                opts.DataZoomOpts(type_="inside")
            ],
            xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=0.2))),
            yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=True, linestyle_opts=opts.LineStyleOpts(opacity=0.2)))
        )

        with output_chart:
            display(line.render_notebook())

    except Exception as e:
        with output_chart:
            print("绘图失败：", e)

x_dropdown.observe(update_chart, names='value')
y_select.observe(update_chart, names='value')

# 显示所有控件
display(upload_button)
display(widgets.HBox([start_row_dropdown, end_row_dropdown]))
display(widgets.HBox([x_dropdown, y_select]))
display(output_chart)


FileUpload(value=(), accept='.csv,.xlsx', button_style='info', description='上传表格')

Output()